In [1]:
from cluster_model import get_dataset, get_transformer, DATASET_NAME, MODEL_NAME, SEED, N_DATASIZE, get_block_out_label, N_BLOCKS
import cluster_model

ds = get_dataset(DATASET_NAME)
model, _ = get_transformer(MODEL_NAME)
shuffled = ds.shuffle(seed=SEED)['train'][:N_DATASIZE]['text']

ds = shuffled
labs = [get_block_out_label(i) for i in range(N_BLOCKS)]

/home/lev/code/research/ai/lattice_cluster_paths/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


running k-means on cuda:0..


[running kmeans]: 10it [00:00, 53.58it/s, center_shift=0.000044, iteration=10, tol=0.000100]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer
Moving model to device:  cuda


In [2]:
import importlib
importlib.reload(cluster_model)

decomp = cluster_model.Decomposer(model, ds, labs, similarity_cutoff=2)
decomp.load()

Finding optimal number of clusters and such clusters for layer blocks.0.hook_resid_post
Trying 50 clusters for layer blocks.0.hook_resid_post
USING DEVICE cuda:0
running k-means on cuda:0..


[running kmeans]: 26it [00:05,  5.16it/s, center_shift=0.000000, iteration=26, tol=0.000100]  


CLUSTER SHAPE torch.Size([50, 512])
Finding optimal number of clusters and such clusters for layer blocks.1.hook_resid_post
Trying 50 clusters for layer blocks.1.hook_resid_post
USING DEVICE cuda:0
running k-means on cuda:0..


[running kmeans]: 24it [00:04,  4.98it/s, center_shift=0.000000, iteration=24, tol=0.000100]  


CLUSTER SHAPE torch.Size([50, 512])
Finding optimal number of clusters and such clusters for layer blocks.2.hook_resid_post
Trying 50 clusters for layer blocks.2.hook_resid_post
USING DEVICE cuda:0
running k-means on cuda:0..


...

CLUSTER SHAPE torch.Size([50, 512])
Finding optimal number of clusters and such clusters for layer blocks.3.hook_resid_post
Trying 50 clusters for layer blocks.3.hook_resid_post
USING DEVICE cuda:0
running k-means on cuda:0..


[running kmeans]: 44it [00:09,  4.85it/s, center_shift=0.000000, iteration=44, tol=0.000100]   


CLUSTER SHAPE torch.Size([50, 512])
Finding optimal number of clusters and such clusters for layer blocks.4.hook_resid_post
Trying 50 clusters for layer blocks.4.hook_resid_post
USING DEVICE cuda:0
running k-means on cuda:0..


[running kmeans]: 39it [00:07,  5.04it/s, center_shift=0.000000, iteration=39, tol=0.000100]   


CLUSTER SHAPE torch.Size([50, 512])
Getting cluster scores for lattice
Getting scores for layers 0 to 1
Getting scores for layers 1 to 2
Getting scores for layers 2 to 3
Getting scores for layers 3 to 4
CLUSTER SCORES [array([[2.36796947, 2.39530803, 2.47232866, ..., 2.07615933, 2.459008  ,
        2.41135878],
       [2.30888491, 2.36441701, 2.42654503, ..., 2.03596475, 2.41157728,
        2.34682514],
       [2.45849513, 2.44546038, 2.5169896 , ..., 2.11990657, 2.53521847,
        2.52458025],
       ...,
       [2.27056408, 2.27634524, 2.33436734, ..., 1.96454975, 2.35964031,
        2.32024298],
       [2.09042534, 2.15741455, 2.21479929, ..., 1.85518916, 2.18831181,
        2.12166687],
       [2.20429331, 2.25827734, 2.31741743, ..., 1.94552177, 2.30126356,
        2.23906315]]), array([[2.20936052, 2.39251457, 2.35359832, ..., 2.13643841, 2.40919887,
        2.200792  ],
       [2.24101464, 2.42538745, 2.38581588, ..., 2.15134823, 2.41928255,
        2.24182589],
       [2.30980

In [3]:
import utils
importlib.reload(utils)
utils.top_k_dag_paths_dynamic(decomp.lattice_scores, k=10_000)

In [ ]:

score_path = [8, 57, 89]
score_path = [10, 63, 89]
score_path = [3, 3, 1]
score_path = [20, 1, 12]
"""
Paths which seem to have meaning

[1, 3, 25]: relational words like "if" "in" "soon"
[1, 3, 10]: relational word but now it is more specific to "to"
"""

to_score = [d[:100] for d in ds][:100]
scores = decomp.score(
    to_score,
    score_path=score_path
)

DS (3, 2816, 512) 2816
(2816, 3, 512) [20, 1, 12]


In [ ]:
import math
import numpy as np

scores_per_token_set = np.array([max(s) for s in scores])
top_args = np.argsort(scores_per_token_set)[::-1]

tokens = [[model.tokenizer.decode(t) for t in model.tokenizer(d)[
    'input_ids']] for d in to_score]
tokens_reord = [tokens[i] for i in top_args]
scores_reord = [scores[i] for i in top_args]
# TODO: SORT!

# TODO sep fun
from circuitsvis.utils.render import RenderedHTML, render
html = render(
    "TextNeuronActivations",
    tokens=tokens_reord,
    activations=[[[[0 if math.isnan(tok * 100) else int(math.exp(tok * 100))]] for tok in s] for s in scores_reord],
    firstDimensionName="Layer",
    secondDimensionName="Neuron",
    firstDimensionLabels=None,
    secondDimensionLabels=None
)

In [ ]:
scores_per_token_set, scores_per_token_set[top_args]

(array([1.31726889, 1.31726889, 1.31726889, 1.31726889, 1.31726889,
        1.31726889, 1.31726889, 1.31726889, 1.31726889, 1.31726889,
        1.31726889, 1.31726889, 1.31726889, 1.31726889, 1.31726889,
        1.31726889, 1.31726889, 1.31726889, 1.31726889, 1.31726889,
        1.31726889, 1.31726889, 1.31726889, 1.31726889, 1.31726889,
        1.31726889, 1.31726889, 1.31726889, 1.31726889, 1.31726889,
        1.31726889, 1.31726889, 1.31726889, 1.31726889, 1.31726889,
        1.31726889, 1.31726889, 1.31726889, 1.31726889, 1.31726889,
        1.31726889, 1.31726889, 1.31726889, 1.31726889, 1.31726889,
        1.31726889, 1.31726889, 1.31726889, 1.31726889, 1.31726889,
        1.31726889, 1.31726889, 1.31726889, 1.31726889, 1.31726889,
        1.31726889, 1.31726889, 1.31726889, 1.31726889, 1.31726889,
        1.31726889, 1.31726889, 1.31726889, 1.31726889, 1.31726889,
        1.31726889, 1.31726889, 1.31726889, 1.31726889, 1.31726889,
        1.31726889, 1.31726889, 1.31726889, 1.31

In [ ]:
from IPython.core.display import display, HTML

display(HTML(str(html)))

/tmp/ipykernel_282569/288767411.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
